In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('db/nba_data.db')

In [ ]:
query = """
        SELECT 
            CASE 
                WHEN draft_team IN ('Washington Bullets', 'Washington Wizards') THEN 'Washington Wizards'
                WHEN draft_team IN ('New Jersey Nets', 'Brooklyn Nets') THEN 'Brooklyn Nets'
                WHEN draft_team IN ('Charlotte Bobcats', 'Charlotte Hornets') THEN 'Charlotte Hornets'
                WHEN draft_team IN ('Vancouver Grizzlies', 'Memphis Grizzlies') THEN 'Memphis Grizzlies'
                WHEN draft_team IN ('New Orleans Hornets', 'New Orleans Pelicans') THEN 'New Orleans Pelicans'
                WHEN draft_team IN ('Kansas City Kings', 'Sacramento Kings') THEN 'Sacramento Kings'
                WHEN draft_team IN ('Seattle SuperSonics', 'Oklahoma City Thunder') THEN 'Oklahoma City Thunder'
                ELSE draft_team
            END AS franchise,
            COUNT(pick) AS number_of_picks
        FROM lottery_data
        GROUP BY franchise
        ORDER BY number_of_picks DESC
        """

df = pd.read_sql_query(query, conn)
df.to_csv("data/num_lottery_picks.csv", index=False)
pd.set_option('display.max_rows', None)
print(df)


                 franchise  number_of_picks
0         Sacramento Kings               28
1        Charlotte Hornets               27
2            Orlando Magic               24
3     Los Angeles Clippers               24
4   Minnesota Timberwolves               23
5    Golden State Warriors               23
6      Cleveland Cavaliers               22
7       Washington Wizards               20
8       Philadelphia 76ers               20
9    Oklahoma City Thunder               19
10         Toronto Raptors               18
11            Phoenix Suns               18
12           Atlanta Hawks               18
13         Milwaukee Bucks               17
14         Detroit Pistons               17
15           Chicago Bulls               17
16       Memphis Grizzlies               16
17          Boston Celtics               16
18        Dallas Mavericks               15
19  Portland Trail Blazers               14
20    New Orleans Pelicans               14
21          Denver Nuggets      

In [ ]:
query2 = """
        WITH top3 AS (
          SELECT draft, pick, CAST(REPLACE(chances, '%', '') AS REAL) AS chances, draft_team
          FROM lottery_data
          WHERE pick IN (1, 2, 3)
        ),

        total_combinations AS (
          SELECT draft, SUM(CAST(REPLACE(chances, '%', '') AS REAL)) AS total_combinations
          FROM lottery_data
          GROUP BY draft
        ),

        pivot_columns AS (
          SELECT t.draft,
          tc.total_combinations,
          MAX(CASE WHEN t.pick = 1 THEN t.chances END) AS c1,
          MAX(CASE WHEN t.pick = 2 THEN t.chances END) AS c2,
          MAX(CASE WHEN t.pick = 3 THEN t.chances END) AS c3,
          MAX(CASE WHEN t.pick = 1 THEN t.draft_team END) AS team1,
          MAX(CASE WHEN t.pick = 2 THEN t.draft_team END) AS team2,
          MAX(CASE WHEN t.pick = 3 THEN t.draft_team END) AS team3  
          FROM top3 t
          JOIN total_combinations tc 
          ON t.draft = tc.draft
          GROUP BY t.draft, tc.total_combinations
        ),

        probabilities AS (
          SELECT draft, 
            ROUND(
              1.0 * c1 / total_combinations *
              c2 / (total_combinations - c1) *
              c3 / (total_combinations - c1 - c2) * 100,
              3
            ) AS top3_probability_percent,
            team1, team2, team3
          FROM pivot_columns
        )

        SELECT * FROM probabilities
        ORDER BY draft DESC;
"""

df2 = pd.read_sql_query(query2, conn)
df2.to_csv("data/top3_probability_results.csv", index=False)
pd.set_option('display.max_rows', None)
print(df2)

                     draft  top3_probability_percent                   team1  \
0   2025 NBA Draft Lottery                     0.013        Dallas Mavericks   
1   2024 NBA Draft Lottery                     0.023           Atlanta Hawks   
2   2023 NBA Draft Lottery                     0.291       San Antonio Spurs   
3   2022 NBA Draft Lottery                     0.388           Orlando Magic   
4   2021 NBA Draft Lottery                     0.364         Detroit Pistons   
5   2020 NBA Draft Lottery                     0.190  Minnesota Timberwolves   
6   2019 NBA Draft Lottery                     0.061    New Orleans Pelicans   
7   2018 NBA Draft Lottery                     0.347            Phoenix Suns   
8   2017 NBA Draft Lottery                     0.245      Philadelphia 76ers   
9   2016 NBA Draft Lottery                     1.878      Philadelphia 76ers   
10  2015 NBA Draft Lottery                     0.981  Minnesota Timberwolves   
11  2014 NBA Draft Lottery              

In [ ]:
query3 = """
        SELECT draft, draft_team, CAST(odds AS REAL) AS odds_real, player_taken, prelottery_position, record
        FROM lottery_data
        WHERE pick == 1 
        ORDER BY odds_real ASC
        LIMIT 10
"""

df3 = pd.read_sql_query(query3, conn)
df3.to_csv("data/top10_unlikely_first.csv", index=False)
pd.set_option('display.max_rows', None)
print(df3)

                    draft              draft_team  odds_real  \
0  1993 NBA Draft Lottery           Orlando Magic       1.52   
1  2014 NBA Draft Lottery     Cleveland Cavaliers       1.70   
2  2008 NBA Draft Lottery           Chicago Bulls       1.70   
3  2025 NBA Draft Lottery        Dallas Mavericks       1.80   
4  2011 NBA Draft Lottery     Cleveland Cavaliers       2.80   
5  2024 NBA Draft Lottery           Atlanta Hawks       3.00   
6  2000 NBA Draft Lottery         New Jersey Nets       4.40   
7  2007 NBA Draft Lottery  Portland Trail Blazers       5.30   
8  2019 NBA Draft Lottery    New Orleans Pelicans       6.00   
9  2005 NBA Draft Lottery         Milwaukee Bucks       6.30   

         player_taken prelottery_position record  
0        Chris Webber                11th  41-41  
1      Andrew Wiggins                 9th  33-49  
2        Derrick Rose                 9th  33-49  
3        Cooper Flagg                11th  39-43  
4        Kyrie Irving                 8t

In [ ]:
query4 = """
        WITH all_teams AS (
            SELECT DISTINCT 
                CASE 
                    WHEN draft_team IN ('Washington Bullets', 'Washington Wizards') THEN 'Washington Wizards'
                    WHEN draft_team IN ('New Jersey Nets', 'Brooklyn Nets') THEN 'Brooklyn Nets'
                    WHEN draft_team IN ('Charlotte Bobcats', 'Charlotte Hornets') THEN 'Charlotte Hornets'
                    WHEN draft_team IN ('Vancouver Grizzlies', 'Memphis Grizzlies') THEN 'Memphis Grizzlies'
                    WHEN draft_team IN ('New Orleans Hornets', 'New Orleans Pelicans') THEN 'New Orleans Pelicans'
                    WHEN draft_team IN ('Kansas City Kings', 'Sacramento Kings') THEN 'Sacramento Kings'
                    WHEN draft_team IN ('Seattle SuperSonics', 'Oklahoma City Thunder') THEN 'Oklahoma City Thunder'
                    ELSE draft_team
                END AS franchise
            FROM lottery_data
        ),

        firsts AS (
            SELECT 
                CASE 
                    WHEN draft_team IN ('Washington Bullets', 'Washington Wizards') THEN 'Washington Wizards'
                    WHEN draft_team IN ('New Jersey Nets', 'Brooklyn Nets') THEN 'Brooklyn Nets'
                    WHEN draft_team IN ('Charlotte Bobcats', 'Charlotte Hornets') THEN 'Charlotte Hornets'
                    WHEN draft_team IN ('Vancouver Grizzlies', 'Memphis Grizzlies') THEN 'Memphis Grizzlies'
                    WHEN draft_team IN ('New Orleans Hornets', 'New Orleans Pelicans') THEN 'New Orleans Pelicans'
                    WHEN draft_team IN ('Kansas City Kings', 'Sacramento Kings') THEN 'Sacramento Kings'
                    WHEN draft_team IN ('Seattle SuperSonics', 'Oklahoma City Thunder') THEN 'Oklahoma City Thunder'
                    ELSE draft_team
                END AS franchise,
                COUNT(*) AS first_pick_count
            FROM lottery_data
            WHERE pick = 1
            GROUP BY franchise
        )

        SELECT 
            a.franchise, 
            COALESCE(f.first_pick_count, 0) AS first_pick_count
        FROM all_teams a
        LEFT JOIN firsts f ON a.franchise = f.franchise
        ORDER BY first_pick_count DESC

"""

df4 = pd.read_sql_query(query4, conn)
df4.to_csv("data/most_firsts.csv", index=False)
pd.set_option('display.max_rows', None)
print(df4)

                 franchise  first_pick_count
0      Cleveland Cavaliers                 5
1            Orlando Magic                 4
2        San Antonio Spurs                 3
3       Philadelphia 76ers                 3
4     Los Angeles Clippers                 3
5       Washington Wizards                 2
6     New Orleans Pelicans                 2
7            Brooklyn Nets                 2
8            Chicago Bulls                 2
9   Minnesota Timberwolves                 2
10         Milwaukee Bucks                 2
11        Dallas Mavericks                 1
12       Charlotte Hornets                 1
13         Toronto Raptors                 1
14         Houston Rockets                 1
15  Portland Trail Blazers                 1
16           Atlanta Hawks                 1
17         Detroit Pistons                 1
18        Sacramento Kings                 1
19         New York Knicks                 1
20   Golden State Warriors                 1
21        

In [15]:
conn.close()